In [1]:
## for the follwing code
#         # Embedding block.
#         x = self.emb(data.atomic_numbers.long(), rbf, i, j)
#         P = self.output_blocks[0](x, rbf, i, num_nodes=pos.size(0))
#         print(x.shape, P.shape)
        
#         ## change
#         def capture_embeddings(module, input, output):
#             self.embs.append(output.detach())  # Store the output embeddings
            
#         ## change
#         hook_handle = self.interaction_blocks[-1].register_forward_hook(capture_embeddings)
            
#         # Interaction blocks.
#         for interaction_block, output_block in zip(
#             self.interaction_blocks, self.output_blocks[1:]
#         ):
#             x = interaction_block(x, rbf, sbf, idx_kj, idx_ji)
#             P += output_block(x, rbf, i, num_nodes=pos.size(0))
#             print(x.shape, P.shape)

## getting following output
# Atoms(symbols='H5Au48Co8O17', pbc=True, cell=[5.878883441, 20.365049623, 20.81515], calculator=OCPCalculator(...))
# torch.Size([3477, 256]) torch.Size([78, 1])
# torch.Size([3477, 256]) torch.Size([78, 1])
# torch.Size([3477, 256]) torch.Size([78, 1])
# torch.Size([3477, 256]) torch.Size([78, 1])
# Atoms(symbols='CH9Au48Co8O17', pbc=True, cell=[5.878883441, 20.365049623, 20.81515], calculator=OCPCalculator(...))
# torch.Size([3533, 256]) torch.Size([83, 1])
# torch.Size([3533, 256]) torch.Size([83, 1])
# torch.Size([3533, 256]) torch.Size([83, 1])
# torch.Size([3533, 256]) torch.Size([83, 1])
# -1.2942085266113281

In [2]:
import os
import numpy as np
import ase.io
from ase.constraints import FixAtoms
from ase.build import add_adsorbate, molecule, surface
from pymatgen.ext.matproj import MPRester
from pymatgen.core.surface import generate_all_slabs, SlabGenerator
from pymatgen.io.ase import AseAtomsAdaptor
from ocpmodels.common.relaxation.ase_utils import OCPCalculator
import torch
import pickle
import json
import pandas as pd

In [4]:
config_yml_path = "ocp/configs/is2re/all/dimenet_plus_plus/dpp.yml"
checkpoint_path = "dimenetpp_all.pt"
# Define the calculator
calc = OCPCalculator(config_yml=config_yml_path, checkpoint=checkpoint_path)

/opt/anaconda3/lib/python3.10/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [5]:
calc

In [6]:
with open('datasets/reactions.pickle', 'rb') as f:
    loaded_reactions = pickle.load(f)

In [7]:
len(loaded_reactions)

88587

In [9]:
# list_rinfo = []
# for ir, r in enumerate(loaded_reactions):
#     try:
#         if ir%1000==0:
#             print(ir, len(list_rinfo))
    
#         ## retrieve slab and adsorbate
#         ase_slab = r['reactionSystems']['star']
#         d_prod = json.loads(r['products'])
#         if len(d_prod.keys()) != 1:
#             continue
#         pkey = list(d_prod.keys())[0].replace('star', '')
#         ads_symbol = pkey    

#         ## get slab energy and embeddings
#         adslab = ase_slab.copy()
#         adslab.calc = calc
#         # print(adslab)
#         slab_energy = adslab.get_potential_energy()
#         slab_embs = torch.mean(calc.trainer.model.module.embs[0], dim=0)    

#         ## get slab + adsorbate energy and embeddings
#         # add the molecule
#         adsorbate = molecule(ads_symbol)
#         add_adsorbate(adslab, adsorbate, 3, offset=(1, 1))
#         # print(adslab)
#         ads_energy = adslab.get_potential_energy()
#         ads_embs = torch.mean(calc.trainer.model.module.embs[0], dim=0)    
        
#         ## additional info
#         sc = r['surfaceComposition']
#         facet = r['facet']
#         pval = d_prod[list(d_prod.keys())[0]]        
#         re = r['reactionEnergy']
#         nre = re/pval
        
#         eqn = r['Equation']
#         ae = r['activationEnergy']
#         cc = r['chemicalComposition']
#         d_cvr = json.loads(r['coverages'])
#         dft_code = r['dftCode']
#         dft_func = r['dftFunctional']
#         pubid = r['pubId']
#         d_reactants = json.loads(r['reactants'])
#         sites = json.loads(r['sites'])
#         username = r['username']

#         rinfo = {
#             'ase_slab': ase_slab,            
#             'd_prod': d_prod,            
#             'pkey': pkey,
#             'slab_embs': slab_embs,
#             'ads_embs': ads_embs,
            
#             'sc': sc,
#             'facet': facet,            
#             'pval': pval,            
#             're': re,
#             'nre': nre,
            
#             'eqn': eqn,
#             'ae': ae, 
#             'cc': cc, 
#             'd_cvr': d_cvr, 
#             'dft_code': dft_code,
#             'dft_func': dft_func,
#             'pubid': pubid,
#             'd_reactants': d_reactants,
#             'sites': sites,
#             'username': username            
#         }
#         list_rinfo.append(rinfo)
#     except Exception as e:
#         pass
    

In [10]:
# with open('datasets/reactions_info_2.pickle', 'wb') as f:
#     pickle.dump(list_rinfo, f)

In [11]:
with open('datasets/reactions_info_2.pickle', 'rb') as f:
    loaded_list_rinfo = pickle.load(f)

In [12]:
len(loaded_list_rinfo)

43678

In [13]:
# Convert tensor values in 'ads_embs' to lists of floats
for item in loaded_list_rinfo:
    item['ads_embs'] = [float(x) for x in item['ads_embs']]

In [14]:
# Convert list of dictionaries to DataFrame
df = pd.DataFrame(loaded_list_rinfo)

In [15]:
# Split 'ads_embs' into separate columns
ads_embs_cols = df['ads_embs'].apply(pd.Series)
ads_embs_cols.columns = ['e' + str(i) for i in range(256)]

In [16]:
# Drop the original 'ads_embs' column and concatenate the new columns
df = df.drop('ads_embs', axis=1)
df = pd.concat([df, ads_embs_cols], axis=1)

In [17]:
df = df.sort_values(by='nre')
df = df.groupby(['sc', 'facet', 'pkey']).first().reset_index()
print(df.shape)

(10938, 275)


In [18]:
df = df[['e' + str(i) for i in range(256)] + ['nre']]

In [19]:
df.shape

(10938, 257)

In [20]:
df.head(2)

,e0,e1,e2,e3,e4,e5,e6,e7,e8,e9,...,e247,e248,e249,e250,e251,e252,e253,e254,e255,nre
0,-0.051727,-0.286628,1.382931,-1.435718,2.597454,-0.519882,1.805875,-1.458474,-1.170391,0.392702,...,-0.281010,-1.516974,-0.710036,2.207847,-0.556048,0.106218,0.500164,0.067475,-1.460815,5.608979
1,-0.174674,-0.420665,1.293232,-1.630182,2.610617,0.041520,1.422713,-1.272242,-0.592301,0.131498,...,-0.154266,-1.227633,-0.841806,2.116062,-0.115456,0.222690,0.624747,0.546530,-1.335673,3.996914


In [21]:
# df.to_pickle('datasets/reactions_info_df_2.pickle')

In [22]:
df = pd.read_pickle('datasets/reactions_info_df_2.pickle')

In [23]:
df.shape

(10938, 257)

In [24]:
df.head()

,e0,e1,e2,e3,e4,e5,e6,e7,e8,e9,...,e247,e248,e249,e250,e251,e252,e253,e254,e255,nre
0,-0.051727,-0.286628,1.382931,-1.435718,2.597454,-0.519882,1.805875,-1.458474,-1.170391,0.392702,...,-0.281010,-1.516974,-0.710036,2.207847,-0.556048,0.106218,0.500164,0.067475,-1.460815,5.608979
1,-0.174674,-0.420665,1.293232,-1.630182,2.610617,0.041520,1.422713,-1.272242,-0.592301,0.131498,...,-0.154266,-1.227633,-0.841806,2.116062,-0.115456,0.222690,0.624747,0.546530,-1.335673,3.996914
2,-0.343314,-0.235674,1.567940,-1.390991,1.977526,0.139984,1.089224,-1.166580,-0.692959,-0.110241,...,-0.341959,-1.198122,-0.903510,1.975405,-0.591985,-0.113284,0.572909,0.548986,-1.033705,1.476339
3,-0.334801,-0.539823,1.276774,-1.881636,2.435134,-0.462364,1.572978,-1.456305,-1.060732,-0.164784,...,-0.248361,-1.591639,-1.124636,2.116420,-0.781833,-0.052015,0.452242,-0.112862,-1.649781,0.338155
4,-0.344107,-0.327844,1.366605,-1.291086,2.136957,-0.279175,1.176640,-1.202859,-0.808488,-0.204060,...,-0.250685,-1.344624,-0.927819,1.914095,-0.652576,-0.078962,0.684331,0.447531,-1.397458,-0.120299
